In [1]:
import pandas as pd 
import time 
import datetime 
import MetaTrader5 as mt5
import  logging
import os

In [2]:
def create_trade_history_df(symbol):
    cols = [    
                "trade_date_id",
                "create_order_date",
                "buy_ticket",
                "sell_ticket",
                "account_balance",
                "account_equity",
                "account_margin",
                "current_market_price",
                "volume",
                "buy_at",
                "sell_at",
                "buy_status",
                "sell_status",
                "fib_number",
                "next_fib_number",
                "next_volume",
                "next_buy_at",
                "next_sell_at",
                "market_direction",
                "close_all_order_type",
                "close_all_when_market_price_at",
                "is_first_order_of_sequence",
            ]
    df_file_name = f"{symbol}_trade_history_df.csv"
    
    # Check if the file already exists
    if os.path.exists(df_file_name):
        print(f"File {df_file_name} already exists. Skipping creation.")
        return

    trade_history_df = pd.DataFrame(columns=cols)
    trade_history_df.to_csv(df_file_name, index=None)
    print(f"Data frame has been created. File name: {df_file_name}")


In [3]:
def make_order(symbol, volume, order_type):
        tick = mt5.symbol_info_tick(symbol)
        if not tick:
            print("Failed to fetch tick for", symbol)
            return None
        
        order_dict = {'buy': 0, 'sell': 1}
        price_dict = {'buy': tick.ask, 'sell': tick.bid}

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_dict[order_type],
            "price": price_dict[order_type],
            "deviation": 20,
            "magic": 100,
            "comment": "python market order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        order_result = mt5.order_send(request)
        if not order_result or order_result.retcode != mt5.TRADE_RETCODE_DONE:
            print("Failed to send order:", order_result.comment if order_result else "Unknown error")
            return None
        print(f"order_result: {order_result}")
        return order_result

In [4]:
def close_order(symbol,type,ticket,volume):
        tick = mt5.symbol_info_tick(symbol)
        if not tick:
            print(f"Failed to fetch tick for {symbol}")
            return None
        if type == "buy":
              order_type = mt5.ORDER_TYPE_SELL
              price = tick.ask
        elif type == "sell":
              order_type = mt5.ORDER_TYPE_BUY
              price = tick.bid

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "position": int(ticket),
            "symbol": symbol,
            "volume": volume,
            "type": order_type,
            "price": price,
            "deviation": 20,
            "magic": 100,
            "comment": "python script close",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)
        if not result or result.retcode != mt5.TRADE_RETCODE_DONE:
            print("Failed to send order. Return code:", result.retcode if result else "No result")
            print("Error:", mt5.last_error())  # This will print the last error from MT5

        return result

In [5]:
def has_open_orders(symbol):
    trade_history_df = pd.read_csv(f"{symbol}_trade_history_df.csv")
    if trade_history_df.empty:
        return False
    
    last_trade_buy_status = trade_history_df["buy_status"].iloc[-1]
    last_trade_sell_status = trade_history_df["sell_status"].iloc[-1]
    print(f"last_trade_buy_status : {last_trade_buy_status}")
    print(f"last_trade_sell_status : {last_trade_sell_status}")
    
    if last_trade_buy_status == "open" and last_trade_sell_status == "open":
        return True
    elif last_trade_buy_status == "close" and last_trade_sell_status == "close":
        return False
    else:
        # Handle other cases here, or raise an exception if these cases shouldn't occur
        print("Unexpected trade status combination.")
        return False  # or raise an exception


In [6]:
def get_current_market_price(symbol):
        tick = mt5.symbol_info_tick(symbol)
        if not tick:
            print(f"Failed to fetch tick for {symbol}")
            return None
        return tick.ask  # or tick.bid based on the requirement

In [7]:
def make_first_order(symbol,volume):
    account_info = mt5.account_info()
    first_fib_number = 1
    next_fib_number = 2
    
    print(f"Account Balance: {account_info.balance}")
    print(f"Account Equity: {account_info.equity}")
    print(f"Account Margin: {account_info.margin}")
    print(f"Account Free Margin: {account_info.margin_free}")
    print(f"fib number :{first_fib_number}")

    symbol_properties = mt5.symbol_info(symbol)
    if symbol_properties:
        print(f"Symbol Margin Required for 1 lot: {symbol_properties.margin_hedged}")
    
    create_trade_history_df(symbol)

    current_market_price = get_current_market_price(symbol)

    make_buy_order = make_order("EURUSD",0.01,"sell")
    make_sell_order = make_order("EURUSD",0.01,"buy")

    next_buy_volume = volume * int(make_buy_order.price)
    next_sell_volume = volume * int(make_sell_order.price)
    
    # Create a new row with the values you want to append
    first_trade_row = {
        "trade_date_id": pd.Timestamp.now().strftime("%d%m%y%H%M%S"),
        "buy_ticket":make_buy_order.order,
        "sell_ticket":make_sell_order.order,
        "create_order_date": pd.Timestamp.now(),  # Current timestamp
        "account_balance": account_info.balance,
        "account_equity": account_info.equity,
        "account_margin": account_info.margin,
        "current_market_price" : current_market_price,
        "buy_volume": make_buy_order.volume,
        "sell_volume": make_sell_order.volume,
        "buy_at": make_buy_order.price,
        "sell_at": make_sell_order.price,
        "buy_status": "open",
        "sell_status": "open",
        "fib_number": first_fib_number,
        "next_fib_number": next_fib_number,
        "next_volume": next_fib_number * volume,
        "next_buy_at": make_buy_order.price + next_buy_volume,
        "next_sell_at": make_sell_order.price - next_sell_volume,
        "market_direction": None,
        "close_all_order_type": None,
        "close_all_when_market_price_at": None,
        "is_first_order_of_sequence": True  # Assuming this is the first order of the sequence
    }
    trade_history_df = pd.read_csv(f"{symbol}_trade_history_df.csv")
    # Append the new row to the DataFrame
    trade_history_df = pd.concat([trade_history_df, pd.DataFrame([first_trade_row])], ignore_index=True)

    # Save the updated DataFrame back to the CSV
    trade_history_df.to_csv(f"{symbol}_trade_history_df.csv", index=False)

    print(f"Data frame has been updated. File name: {symbol}_trade_history_df.csv")

In [8]:
def close_profit_order_and_make_next_buy_order(symbol, market_direction):
    trade_history_df = pd.read_csv(f"{symbol}_trade_history_df.csv")
    account_info = mt5.account_info()

    close_order("")
    
    make_order("EURUSD",volume = 0.01,order_type="sell")
    make_order("EURUSD",volume = 0.01,order_type="buy")

In [9]:
def run(symbol,volume):
    print(f"PROGRAM START")
    create_trade_history_df(symbol)
    is_has_open_orders = has_open_orders(symbol)
    print(is_has_open_orders)
    
    if is_has_open_orders == False:
        print(f"Make first trade")
        make_first_order(symbol,volume)
    
    elif is_has_open_orders == True:
        print(f"Already has opened order")

    while True:
        trade_history_df = pd.read_csv(f"{symbol}_trade_history_df.csv")
        current_market_price = get_current_market_price(symbol)
        next_buy_at = trade_history_df["next_buy_at"].iloc[-1]
        next_sell_at = trade_history_df["next_sell_at"].iloc[-1]
        print(f"current_market_price: {current_market_price}")
        print(next_buy_at,next_sell_at)
        
        if current_market_price >= next_buy_at: 
            print(f"Close current buy order")
            buy_ticket = trade_history_df["buy_ticket"].iloc[-1]
            buy_volume = trade_history_df["buy_volume"].iloc[-1]
            close_order(symbol,"buy",buy_ticket,buy_volume)
        
        elif current_market_price <= next_sell_at: 
            print(f"Close current sell order")
            sell_ticket = trade_history_df["sell_ticket"].iloc[-1]
            sell_volume = trade_history_df["sell_volume"].iloc[-1]
            print(f"sell_ticket: {sell_ticket}")
            print(f"sell_volume: {sell_volume}")
            close_order(symbol,"sell",sell_ticket,sell_volume)

        elif current_market_price >= next_sell_at and current_market_price <= next_buy_at:
            print(f"Keep holding")

        print(f"all position: {mt5.positions_get(symbol=symbol)}")
        time.sleep(20)
    

In [10]:
# OrderSendResult(retcode=10009, deal=372056434, order=511009703, volume=0.01, price=1.05561, bid=1.05561, ask=1.05561, comment='Request executed', request_id=1145893598, retcode_external=0, request=TradeRequest(action=1, magic=100, order=0, symbol='EURUSD', volume=0.01, price=1.05561, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))
# OrderSendResult(retcode=10009, deal=372056437, order=511009707, volume=0.01, price=1.05563, bid=1.05562, ask=1.05563, comment='Request executed', request_id=1145893599, retcode_external=0, request=TradeRequest(action=1, magic=100, order=0, symbol='EURUSD', volume=0.01, price=1.05561, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))
# all position: (TradePosition(ticket=511009703, time=1698359059, time_msc=1698359059741, time_update=1698359059, time_update_msc=1698359059741, type=1, magic=100, identifier=511009703, reason=3, volume=0.01, price_open=1.05561, sl=0.0, tp=0.0, price_current=1.05533, swap=0.0, profit=0.28, symbol='EURUSD', comment='python market or', external_id=''), TradePosition(ticket=511009707, time=1698359060, time_msc=1698359060321, time_update=1698359060, time_update_msc=1698359060321, type=0, magic=100, identifier=511009707, reason=3, volume=0.01, price_open=1.05563, sl=0.0, tp=0.0, price_current=1.05532, swap=0.0, profit=-0.31, symbol='EURUSD', comment='python market or', external_id=''))

In [11]:
mt5.initialize(login=51446835, server="ICMarketsSC-Demo", password="qwfgKZdZ")
run("EURUSD",volume = 0.01)

PROGRAM START
Data frame has been created. File name: EURUSD_trade_history_df.csv
False
Make first trade
Account Balance: 525991.74
Account Equity: 525991.74
Account Margin: 0.0
Account Free Margin: 525991.74
fib number :1
Symbol Margin Required for 1 lot: 0.0
File EURUSD_trade_history_df.csv already exists. Skipping creation.
order_result: OrderSendResult(retcode=10009, deal=372103733, order=511065287, volume=0.01, price=1.05593, bid=1.05593, ask=1.05593, comment='Request executed', request_id=1145893643, retcode_external=0, request=TradeRequest(action=1, magic=100, order=0, symbol='EURUSD', volume=0.01, price=1.05592, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))
order_result: OrderSendResult(retcode=10009, deal=372103735, order=511065289, volume=0.01, price=1.05593, bid=1.05593, ask=1.05593, comment='Request executed', request_id=1145893644, retcode_external=0, request=TradeR

C:\Users\Pham Quoc Bao\AppData\Local\Temp\ipykernel_10264\1238014821.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_history_df = pd.concat([trade_history_df, pd.DataFrame([first_trade_row])], ignore_index=True)


current_market_price: 1.05592
1.06593 1.04593
Close current sell order
sell_ticket: 511065289
sell_volume: 0.01
Failed to send order. Return code: 10013
Error: (1, 'Success')
all position: (TradePosition(ticket=511065287, time=1698363509, time_msc=1698363509819, time_update=1698363509, time_update_msc=1698363509819, type=1, magic=100, identifier=511065287, reason=3, volume=0.01, price_open=1.05593, sl=0.0, tp=0.0, price_current=1.05592, swap=0.0, profit=0.01, symbol='EURUSD', comment='python market or', external_id=''), TradePosition(ticket=511065289, time=1698363510, time_msc=1698363510382, time_update=1698363510, time_update_msc=1698363510382, type=0, magic=100, identifier=511065289, reason=3, volume=0.01, price_open=1.05593, sl=0.0, tp=0.0, price_current=1.05592, swap=0.0, profit=-0.01, symbol='EURUSD', comment='python market or', external_id=''))
current_market_price: 1.05592
1.06593 1.04593
Close current sell order
sell_ticket: 511065289
sell_volume: 0.01
Failed to send order. Ret